In [ ]:
import time
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests as rq
import re
from time import sleep
from random import randint
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from IPython.core.display import clear_output

##depends on which website you going
page = rq.get("https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=&distance_to=0.0&price_from=0&price_to=&q=tanjong+pagar+cafe&ll=&commit=Apply")
#download the website, to know if succcessful, write 'page' then run
#the status should come out 200

# I used Chrome; you can use whichever browser you like.
driver = webdriver.Chrome()

# Tell Selenium to get the URL you're interested in.
driver.get("https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=&distance_to=0.0&price_from=0&price_to=&q=tanjong+pagar+cafe&ll=&commit=Apply")


lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
                lastCount = lenOfPage
                time.sleep(3)
                lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                time.sleep(6)
                if lastCount==lenOfPage:
                    match=True

#Now that the page is fully scrolled, grab the source code.
source_data = driver.page_source

#Throw the source into BeautifulSoup and start parsing
soup = bs(source_data,'lxml')


### xxxxx 
### below codes can just add onto 'final burpple individual cafes' code , since the scrolling of the page is the same
### the above codes is irrelevant if you already parse the website 

container = soup.find_all("div",{"class": "searchVenue-header card-item card-item--header"})
    

links = []
price = []
food = []
review = []
names = []

for item in container:
        
    review_elem = item.find("span",{"class": "searchVenue-header-reviews"}).text
    
#you can only get .text from find()
#if you want to get from findall() you need to specify the index eg. [0]

   #must put .text if not the review_elem will be the html code instead of specific text you want
    reviewstrip = review_elem.strip('Reviews": ')
    intreview = int(reviewstrip)
        
        
    if (intreview > 4 ):
    #in case some cafes dont have this information
        link_elem = item.find('a', attrs={'href': re.compile("^/")})
    #re.compile("^/") is to match the url starting with /
        links.append(link_elem.get('href'))
        
        names_elem = item.find("span",{"class": "searchVenue-header-name-name headingMedium"}).text
        names.append(names_elem)
        

In [ ]:
testlink = pd.DataFrame({
        "Cafe_Name ": names,
        "Links" : links
    
    })

testlink

In [ ]:
ex = pd.ExcelWriter('CompiledCafeLinks.xlsx', options={'encoding':'utf-8'})
testlink.to_excel(ex, 'Data', index=False)
ex.save()

In [ ]:
##Save time for subsequent scripts, so dont need parse the burpple tanjong pagar cafes again to find the links

import pandas as pd

xl_workbook = pd.ExcelFile('C:/Users/Mei Xuan/Downloads/CompiledCafeLinks.xlsx')  # Load the excel workbook
df = xl_workbook.parse("Data")  # Parse the sheet into a dataframe
aList = df['Links'].tolist()

In [ ]:
##Test if the link will work
print("Test Next Page URL: {}".format('https://www.burpple.com'+aList[0]))